In [16]:
import numpy as np
import win32gui, win32ui, win32con
from threading import Thread, Lock
from windowcapture import WindowCapture
import cv2
import os

In [2]:
# Cargar la imagen en la que quieres buscar la otra imagen
imagen_principal = cv2.imread('Screenshot.jpg')

# Cargar la imagen que quieres buscar dentro de la imagen principal
imagen_a_buscar = cv2.imread('buscar.jpg')

# Buscar la imagen en la imagen principal
resultado = cv2.matchTemplate(imagen_principal, imagen_a_buscar, cv2.TM_CCOEFF_NORMED)

# Definir un umbral para la confianza de la coincidencia
umbral = 0.8

# Encontrar las ubicaciones donde la confianza de coincidencia es mayor que el umbral
ubicaciones = np.where(resultado >= umbral)

# Dibujar un rectángulo verde alrededor de cada ubicación encontrada
for punto in zip(*ubicaciones[::-1]):
    inicio = punto
    fin = (inicio[0] + imagen_a_buscar.shape[1], inicio[1] + imagen_a_buscar.shape[0])
    cv2.rectangle(imagen_principal, inicio, fin, (0, 255, 0), 2)

# Mostrar la imagen con los rectángulos dibujados
cv2.imshow('Imagen con Rectángulos', imagen_principal)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
wincap = WindowCapture("EVE - Guileo Galo")
while(True):
    screenshot = wincap.get_screenshot()
    # Convertir la captura de pantalla a una imagen OpenCV
    #screenshot_cv = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
    cv2.imshow('eveonline', screenshot)
    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows()
        break

In [18]:
def list_window_names():
    def winEnumHandler(hwnd,ctx):
        if win32gui.IsWindowVisible(hwnd):
            print(hex(hwnd),win32gui.GetWindowText(hwnd))
    win32gui.EnumWindows(winEnumHandler,None)

In [19]:
list_window_names()

0x100b2 
0x10616 test_screen_class_wc.ipynb - Untitled (Workspace) - Visual Studio Code
0x906d4 EVE - Guileo Galo
0x104f6 Ver Sony Channel en VIVO Online Por internet GRATIS - TV Libre - Brave
0x10810 EVE Online Launcher
0x6084c Notebooks
0x206ea Ofertas especiales
0x1806a2 Steam
0x200a4 Binance
0x1049c Configuración
0x703fa Configuración
0x2047e 
0x20464 Calculadora
0x20214 Calculadora
0x1018e 
0x10186 
0x1016c 
0x1016a 
0x30086 Microsoft Text Input Application
0x105b4 GitHub Desktop
0x805f4 Anaconda Navigator
0x10130 Program Manager


In [10]:
hwnd = win32gui.FindWindow(None,'EVE - Guileo Galo')
hwnd

591572

In [20]:
window_rect = win32gui.GetWindowRect(hwnd)
window_rect

(-8, -8, 1928, 1048)

In [21]:
w = window_rect[2] - window_rect[0]
h = window_rect[3] - window_rect[1]
print(f' {w} , {h}')

 1936 , 1056


In [22]:
wDC = win32gui.GetWindowDC(hwnd)
wDC

151065274

In [23]:
dcObj = win32ui.CreateDCFromHandle(wDC)
dcObj

object 'PyCDC' - assoc is 000001AA8FD5D1A8, vi=<None>

In [24]:
cDC = dcObj.CreateCompatibleDC()
cDC

object 'PyCDC' - assoc is 000001AA8FD63AD8, vi=<None>

In [25]:
dataBitMap = win32ui.CreateBitmap()
dataBitMap

object 'PyCBitmap' - assoc is 000001AA8FF94810, vi=<None>

In [26]:
dataBitMap.CreateCompatibleBitmap(dcObj, w, h)

In [27]:
cDC.SelectObject(dataBitMap)

object 'PyCBitmap' - assoc is 000001AA8EC98518, vi=<None>

***

In [19]:
wincap = WindowCapture('EVE - Guileo Galo')

***

In [28]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import cv2 as cv
import os

class WindowCapture:
    w = 0
    h = 0
    hwnd = None

    def __init__(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.frombuffer(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[..., :3]  # Eliminar el canal alfa
        img = np.ascontiguousarray(img)

        return img

    def get_window_size(self):
        return self.w, self.h

In [29]:
wincap = WindowCapture("EVE Online Launcher")
while True:
    screenshot = wincap.get_screenshot()
    screenshot = cv.cvtColor(screenshot, cv.COLOR_RGB2BGR)  # Convertir de RGB a BGR
    cv.imshow('eveonline', screenshot)
    key = cv.waitKey(1)
    if key == ord('q'):
        cv.destroyAllWindows()
        break

In [10]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import cv2 as cv
import os

class WindowCapture:
    w = 0
    h = 0
    hwnd = None

    def __init__(self, window_name):
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.frombuffer(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[..., :3]  # Eliminar el canal alfa
        img = np.ascontiguousarray(img)

        # Verificar si la imagen es negra
        if np.sum(img) == 0:
            print("¡La imagen capturada es completamente negra! La detección se realizó después de capturar la imagen.")

        return img

    def get_window_size(self):
        return self.w, self.h

# Ejemplo de uso
wincap = WindowCapture("EVE Online Launcher")
while True:
    screenshot = wincap.get_screenshot()
    cv.imshow('eveonline', screenshot)
    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        break


error: CreateCompatibleDC failed